### 데이터 비율 확인

In [2]:
import os

# 데이터 디렉토리 경로 설정
data_dir = '/home/a202192020/딥러닝/감자병해충/[T원천]01.감자'

# 클래스별 데이터 개수를 저장할 딕셔너리
class_counts = {}

# 각 클래스 디렉토리에서 이미지 파일 개수 세기
for class_name in os.listdir(data_dir):
    class_path = os.path.join(data_dir, class_name)
    if os.path.isdir(class_path):  # 디렉토리인지 확인
        image_count = len([file for file in os.listdir(class_path) if file.endswith('.jpg')])
        class_counts[class_name] = image_count

# 결과 출력
for class_name, count in class_counts.items():
    print(f"Class: {class_name}, Image Count: {count}")


Class: 2.해충, Image Count: 1678
Class: 3.충해, Image Count: 387
Class: 0.정상, Image Count: 7567
Class: 9.증강_(2), Image Count: 8980
Class: 9.증강_(1), Image Count: 11670


json파일을 이용하여 전처리에 활용 (json을 이미지와 매칭), 학습에는 json 사용하지 않음

In [4]:
import os
import json
import shutil

# 경로 설정
json_dir = '/home/a202192020/딥러닝/감자병해충/라벨링'  # JSON 파일의 최상위 폴더 경로
image_dir = '/home/a202192020/딥러닝/감자병해충/[T원천]01.감자'  # 이미지 파일 경로
output_dir = '/home/a202192020/딥러닝/감자병해충/Processed_Data'  # 전처리된 데이터를 저장할 폴더

# 출력 폴더 생성
os.makedirs(output_dir, exist_ok=True)

# 매칭된 파일 개수
matched_count = 0

# JSON 파일 처리
for root, _, files in os.walk(json_dir):
    for json_file in files:
        if json_file.endswith('.json'):
            json_path = os.path.join(root, json_file)
            with open(json_path, 'r', encoding='utf-8') as f:
                try:
                    # JSON 내용 로드
                    data = json.load(f)
                    
                    # 이미지 파일 이름 추출
                    image_filename = data["description"]["image"]
                    
                    # 이미지 파일 실제 경로 찾기
                    image_path = None
                    for dirpath, _, image_files in os.walk(image_dir):
                        if image_filename in image_files:
                            image_path = os.path.join(dirpath, image_filename)
                            break
                    
                    # 클래스 이름 추출 (기본은 JSON 파일의 상위 디렉토리 이름)
                    class_name = os.path.basename(os.path.dirname(json_path))
                    
                    # 정상 데이터를 처리
                    if "정상" in class_name:
                        class_name = "정상"

                    # 증강 데이터는 추가적으로 어떤 데이터를 증강했는지 확인
                    elif "증강" in class_name:
                        parent_dir = os.path.basename(os.path.dirname(root))
                        if "충해" in parent_dir:
                            class_name = "충해_증강"
                        elif "해충" in parent_dir:
                            class_name = "해충_증강"
                        else:
                            class_name = "증강_기타"

                    # 이미지 파일 복사
                    if image_path and os.path.exists(image_path):
                        # 클래스별 폴더 생성
                        label_dir = os.path.join(output_dir, class_name)
                        os.makedirs(label_dir, exist_ok=True)
                        
                        # 이미지 복사
                        shutil.copy(image_path, label_dir)
                        matched_count += 1
                except Exception as e:
                    print(f"Error processing {json_path}: {e}")

print(f"전처리 완료! 매칭된 이미지 파일 수: {matched_count}")


전처리 완료! 매칭된 이미지 파일 수: 36883


In [1]:
import os

# Processed_Data 경로 설정
processed_data_dir = '/home/a202192020/딥러닝/감자병해충/Processed_Data'

# 클래스별 데이터 개수를 저장할 딕셔너리
class_counts = {}

# 각 클래스 디렉토리에서 이미지 파일 개수 세기
for class_name in os.listdir(processed_data_dir):
    class_path = os.path.join(processed_data_dir, class_name)
    if os.path.isdir(class_path):  # 폴더인지 확인
        image_count = len([file for file in os.listdir(class_path) if file.lower().endswith(('.jpg', '.jpeg', '.png'))])
        class_counts[class_name] = image_count

# 결과 출력
for class_name, count in class_counts.items():
    print(f"Class: {class_name}, Image Count: {count}")


Class: [T라벨링]01.감자_3.충해, Image Count: 388
Class: 증강_기타, Image Count: 21950
Class: 정상, Image Count: 12738
Class: [T라벨링]01.감자_2.해충, Image Count: 1807


데이터 불균형을 해결하기 위해 증강_기타 데이터 판별(예시로 3개만 확인)

In [10]:
import os
import json
import random

# 경로 설정
json_dir = '/home/a202192020/딥러닝/감자병해충/라벨링'  # JSON 파일 경로
image_dir = '/home/a202192020/딥러닝/감자병해충/Processed_Data/증강_기타'  # 증강_기타 이미지 경로

# 증강_기타 이미지에서 랜덤으로 샘플 추출
random_sample = random.sample(
    [file for file in os.listdir(image_dir) if file.lower().endswith(('.jpg', '.jpeg', '.png'))], 3
)

# 랜덤으로 선택된 이미지 처리
for image_file in random_sample:
    print(f"\n[이미지 파일]: {image_file}")
    json_matched = False
    for root, _, files in os.walk(json_dir):
        for json_file in files:
            if json_file.endswith('.json'):
                json_path = os.path.join(root, json_file)
                with open(json_path, 'r', encoding='utf-8') as f:
                    try:
                        data = json.load(f)
                        if data["description"]["image"] == image_file:
                            print(f"[JSON 파일]: {json_path}")
                            original_image = data["description"].get("original", None)
                            if original_image:
                                print(f"[원본 이미지 파일]: {original_image}")
                                # 원본 이미지의 클래스 찾기
                                for orig_root, _, orig_files in os.walk(json_dir):
                                    for orig_json in orig_files:
                                        if orig_json.endswith('.json'):
                                            orig_json_path = os.path.join(orig_root, orig_json)
                                            with open(orig_json_path, 'r', encoding='utf-8') as orig_f:
                                                orig_data = json.load(orig_f)
                                                if orig_data["description"]["image"] == original_image:
                                                    found_class = os.path.basename(os.path.dirname(orig_json_path))
                                                    print(f"[원래 클래스]: {found_class}")
                                                    json_matched = True
                                                    break
                                    if json_matched:
                                        break
                        if json_matched:
                            break
                    except Exception as e:
                        print(f"Error processing {json_path}: {e}")
        if json_matched:
            break
    if not json_matched:
        print("[JSON 파일]: 매칭되는 JSON 파일을 찾을 수 없습니다.")



[이미지 파일]: V006_78_3_19_01_03_12_3_6813q_20200918_8_a0003.jpg
[JSON 파일]: /home/a202192020/딥러닝/감자병해충/라벨링/Training/[T라벨링]01.감자_9.증강/V006_78_3_19_01_03_12_3_6813q_20200918_8_a0003.jpg.json
[원본 이미지 파일]: V006_78_3_19_01_03_12_3_6813q_20200918_8.jpg
[원래 클래스]: [T라벨링]01.감자_3.충해

[이미지 파일]: V006_78_3_19_01_03_12_2_1097q_20201015_90_a0005.jpg
[JSON 파일]: /home/a202192020/딥러닝/감자병해충/라벨링/Training/[T라벨링]01.감자_9.증강/V006_78_3_19_01_03_12_2_1097q_20201015_90_a0005.jpg.json
[원본 이미지 파일]: V006_78_3_19_01_03_12_2_1097q_20201015_90.jpg
[원래 클래스]: [T라벨링]01.감자_3.충해

[이미지 파일]: V006_78_2_19_01_07_32_0_0719y_20201022_5_a0000.jpeg
[JSON 파일]: /home/a202192020/딥러닝/감자병해충/라벨링/Training/[T라벨링]01.감자_9.증강/V006_78_2_19_01_07_32_0_0719y_20201022_5_a0000.jpeg.json
[원본 이미지 파일]: V006_78_2_19_01_07_32_0_0719y_20201022_5.jpeg
[원래 클래스]: [T라벨링]01.감자_2.해충


전부 분류

In [11]:
import os
import json
import shutil

# 경로 설정
json_dir = '/home/a202192020/딥러닝/감자병해충/라벨링'  # JSON 파일 경로
image_dir = '/home/a202192020/딥러닝/감자병해충/Processed_Data/증강_기타'  # 증강_기타 이미지 경로
output_dir = '/home/a202192020/딥러닝/감자병해충/Processed_Data'  # 결과 저장 경로

# 결과 저장 딕셔너리
class_origin_counts = {}

# 증강_기타 내 이미지 파일 처리
for image_file in os.listdir(image_dir):
    if image_file.lower().endswith(('.jpg', '.jpeg', '.png')):
        for root, _, files in os.walk(json_dir):
            for json_file in files:
                if json_file.endswith('.json'):
                    json_path = os.path.join(root, json_file)
                    with open(json_path, 'r', encoding='utf-8') as f:
                        try:
                            data = json.load(f)
                            if data["description"]["image"] == image_file:
                                # "original" 필드에서 원본 파일 이름 확인
                                original_image = data["description"].get("original", None)
                                if original_image:
                                    # 원본 이미지의 클래스 찾기
                                    found_class = None
                                    for orig_root, _, orig_files in os.walk(json_dir):
                                        for orig_json in orig_files:
                                            if orig_json.endswith('.json'):
                                                orig_json_path = os.path.join(orig_root, orig_json)
                                                with open(orig_json_path, 'r', encoding='utf-8') as orig_f:
                                                    orig_data = json.load(orig_f)
                                                    if orig_data["description"]["image"] == original_image:
                                                        # 원본 클래스 추출
                                                        found_class = os.path.basename(os.path.dirname(orig_json_path))
                                                        break
                                        if found_class:
                                            break
                                    
                                    # 원본 클래스에 따라 분류
                                    if found_class:
                                        class_origin_counts[found_class] = class_origin_counts.get(found_class, 0) + 1
                                        target_dir = os.path.join(output_dir, found_class)
                                        os.makedirs(target_dir, exist_ok=True)
                                        shutil.copy(os.path.join(image_dir, image_file), target_dir)
                        except Exception as e:
                            print(f"Error processing {json_path}: {e}")

# 결과 출력
print("증강_기타 데이터의 원래 클래스 분포:")
for class_name, count in class_origin_counts.items():
    print(f"{class_name}: {count}")


증강_기타 데이터의 원래 클래스 분포:
[T라벨링]01.감자_2.해충: 18070
[T라벨링]01.감자_3.충해: 3880


In [13]:
import os

# Processed_Data 경로 설정
processed_data_dir = '/home/a202192020/딥러닝/감자병해충/Processed_Data'

# 클래스별 데이터 개수를 저장할 딕셔너리
class_counts = {}

# 각 클래스 디렉토리에서 이미지 파일 개수 세기
for class_name in os.listdir(processed_data_dir):
    class_path = os.path.join(processed_data_dir, class_name)
    if os.path.isdir(class_path):  # 폴더인지 확인
        image_count = len([file for file in os.listdir(class_path) if file.lower().endswith(('.jpg', '.jpeg', '.png'))])
        class_counts[class_name] = image_count

# 결과 출력
for class_name, count in class_counts.items():
    print(f"Class: {class_name}, Image Count: {count}")


Class: [T라벨링]01.감자_3.충해, Image Count: 4268
Class: 증강_기타, Image Count: 21950
Class: 정상, Image Count: 12738
Class: [T라벨링]01.감자_2.해충, Image Count: 19877


증강_기타 데이터는 클래스가 명확하지 않기 때문에 활용하지 않음. 남은 데이터들을 정상/비정상으로 분류함. 

In [ ]:
import os

# Processed_Data 경로 설정
processed_data_dir = '/home/a202192020/딥러닝/감자병해충/Processed_Data'

# 클래스별 데이터 개수를 저장할 딕셔너리
class_counts = {}

# 각 클래스 디렉토리에서 이미지 파일 개수 세기
for class_name in os.listdir(processed_data_dir):
    class_path = os.path.join(processed_data_dir, class_name)
    if os.path.isdir(class_path):  # 폴더인지 확인
        image_count = len([file for file in os.listdir(class_path) if file.lower().endswith(('.jpg', '.jpeg', '.png'))])
        class_counts[class_name] = image_count

# 결과 출력
for class_name, count in class_counts.items():
    print(f"Class: {class_name}, Image Count: {count}")


Class: [T라벨링]01.감자_3.충해, Image Count: 4268
Class: 증강_기타, Image Count: 21950
Class: 정상, Image Count: 12738
Class: [T라벨링]01.감자_2.해충, Image Count: 19877


In [2]:
import os
from PIL import Image

# 데이터 디렉토리 설정
data_dir = '/home/a202192020/딥러닝/감자병해충/Processed_Data'
problematic_dir = '/home/a202192020/딥러닝/감자병해충/Problematic_Files'

# 손상된 파일 이동 디렉토리 생성
os.makedirs(problematic_dir, exist_ok=True)

# 손상된 파일 및 비이미지 파일 찾기 및 이동
def move_problematic_files(directory, output_directory):
    for root, _, files in os.walk(directory):
        for file in files:
            file_path = os.path.join(root, file)
            if file.lower().endswith(('.jpg', '.jpeg', '.png')):
                try:
                    with Image.open(file_path) as img:
                        img.verify()  # 이미지가 열리는지 검증
                except (IOError, SyntaxError):
                    print(f"Moving corrupted image: {file_path}")
                    os.rename(file_path, os.path.join(output_directory, os.path.basename(file_path)))
            else:
                print(f"Moving non-image file: {file_path}")
                os.rename(file_path, os.path.join(output_directory, os.path.basename(file_path)))

move_problematic_files(data_dir, problematic_dir)
print("손상된 파일 및 비이미지 파일 이동 완료!")


Moving corrupted image: /home/a202192020/딥러닝/감자병해충/Processed_Data/[T라벨링]01.감자_3.충해/V006_78_3_19_01_03_12_3_6813q_20201015_93.jpg
손상된 파일 및 비이미지 파일 이동 완료!


In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam

# 다중 GPU 전략 설정
strategy = tf.distribute.MirroredStrategy()
print(f"사용 가능한 GPU 수: {strategy.num_replicas_in_sync}")

# 데이터 경로 설정
data_dir = '/home/a202192020/딥러닝/감자병해충/[T원천]01.감자'

# 데이터 증강 및 전처리
datagen = ImageDataGenerator(
    rescale=1.0/255,  # 데이터 정규화
    validation_split=0.2  # 80% 학습, 20% 검증
)

# 학습 데이터 로더
train_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(64, 64),  # 작은 이미지 크기로 속도 향상
    batch_size=512,        # 병렬 처리를 위해 배치 크기 증가
    class_mode='categorical',  # 다중 클래스 분류
    subset='training',
    shuffle=True,
    classes=['0.정상', '2.해충', '3.충해']  # 사용할 클래스 명시
)

# 검증 데이터 로더
val_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(64, 64),
    batch_size=512,
    class_mode='categorical',
    subset='validation',
    shuffle=True,
    classes=['0.정상', '2.해충', '3.충해']  # 사용할 클래스 명시
)

# 클래스별 데이터 개수
class_counts = {
    '0.정상': 7567,
    '2.해충': 1678,
    '3.충해': 387
}

# 클래스 가중치 계산
total_samples = sum(class_counts.values())
class_weights = {
    0: total_samples / class_counts['0.정상'],  # 정상
    1: total_samples / class_counts['2.해충'],  # 해충
    2: total_samples / class_counts['3.충해']   # 충해
}

print("클래스 가중치:", class_weights)

# 모델 정의
with strategy.scope():  # 다중 GPU 병렬 처리
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),  # 과적합 방지
        Dense(3, activation='softmax')  # 클래스가 3개이므로 softmax 사용
    ])

    model.compile(
        optimizer=Adam(learning_rate=0.001),
        loss='categorical_crossentropy',  # 다중 클래스 손실 함수
        metrics=['accuracy']
    )

# 모델 학습
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=5,  # 빠르게 끝내기 위해 에폭 최소화
    class_weight=class_weights  # 클래스 가중치 적용
)

# 모델 저장
model_save_path = '/home/a202192020/딥러닝/감자병해충/model_multiclass_classification_weighted_parallel.h5'
model.save(model_save_path)
print(f"모델이 저장되었습니다: {model_save_path}")


2024-12-30 05:13:18.096557: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-12-30 05:13:18.209988: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-30 05:13:18.818693: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2024-12-30 05:13:18.818768: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] 

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:GPU:3')
사용 가능한 GPU 수: 4
Found 11948 images belonging to 3 classes.
Found 2985 images belonging to 3 classes.
클래스 가중치: {0: 1.272895467160037, 1: 5.740166865315852, 2: 24.88888888888889}
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


2024-12-30 05:13:58.257441: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:784] AUTO sharding policy will apply DATA sharding policy as it failed to apply FILE sharding policy because of the following reason: Found an unshardable source dataset: name: "TensorDataset/_1"
op: "TensorDataset"
input: "Placeholder/_0"
attr {
  key: "Toutput_types"
  value {
    list {
      type: DT_INT32
    }
  }
}
attr {
  key: "_cardinality"
  value {
    i: 1
  }
}
attr {
  key: "metadata"
  value {
    s: "\n\017TensorDataset:0"
  }
}
attr {
  key: "output_shapes"
  value {
    list {
      shape {
      }
    }
  }
}
experimental_type {
  type_id: TFT_PRODUCT
  args {
    type_id: TFT_DATASET
    args {
      type_id: TFT_PRODUCT
      args {
        type_id: TFT_TENSOR
        args {
          type_id: TFT_INT32
        }
      }
    }
  }
}



Epoch 1/5
INFO:tensorflow:batch_all_reduce: 8 all-reduces with algorithm = nccl, num_packs = 1
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:batch_all_reduce: 8 all-reduces with algorithm = nccl, num_packs = 1
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task

2024-12-30 05:14:35.565022: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8800
2024-12-30 05:14:35.590871: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8800
2024-12-30 05:14:35.622478: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8800
2024-12-30 05:14:35.655005: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8800
2024-12-30 05:14:35.692345: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-12-30 05:14:36.234559: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:630] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2024-12-30 05:14:37.912598: I tensorflow/compiler/xla/service/service.cc:173] XLA service 0x37ebdce0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-12-30 05:1

24/24 [==============================] - ETA: 0s - loss: 2.5355 - accuracy: 0.6625 

2024-12-30 05:26:39.504946: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:784] AUTO sharding policy will apply DATA sharding policy as it failed to apply FILE sharding policy because of the following reason: Found an unshardable source dataset: name: "TensorDataset/_1"
op: "TensorDataset"
input: "Placeholder/_0"
attr {
  key: "Toutput_types"
  value {
    list {
      type: DT_INT32
    }
  }
}
attr {
  key: "_cardinality"
  value {
    i: 1
  }
}
attr {
  key: "metadata"
  value {
    s: "\n\020TensorDataset:41"
  }
}
attr {
  key: "output_shapes"
  value {
    list {
      shape {
      }
    }
  }
}
experimental_type {
  type_id: TFT_PRODUCT
  args {
    type_id: TFT_DATASET
    args {
      type_id: TFT_PRODUCT
      args {
        type_id: TFT_TENSOR
        args {
          type_id: TFT_INT32
        }
      }
    }
  }
}



24/24 [==============================] - 927s 39s/step - loss: 2.5355 - accuracy: 0.6625 - val_loss: 0.3692 - val_accuracy: 0.8791
Epoch 2/5
24/24 [==============================] - 891s 37s/step - loss: 1.5665 - accuracy: 0.8487 - val_loss: 0.5255 - val_accuracy: 0.7467
Epoch 3/5
24/24 [==============================] - 888s 37s/step - loss: 1.2470 - accuracy: 0.8785 - val_loss: 0.2816 - val_accuracy: 0.8640
Epoch 4/5
24/24 [==============================] - 896s 38s/step - loss: 1.0903 - accuracy: 0.8955 - val_loss: 0.4562 - val_accuracy: 0.7913
Epoch 5/5
24/24 [==============================] - 893s 37s/step - loss: 0.9417 - accuracy: 0.9151 - val_loss: 0.2973 - val_accuracy: 0.8700
모델이 저장되었습니다: /home/a202192020/딥러닝/감자병해충/model_multiclass_classification_weighted_parallel.h5
